In [59]:
import requests
import pandas as pd
import json
from dateutil import parser

In [5]:
API_KEY = "2c3aefdf275a880709916f968cc2e719-a8af221b3f9295362c08628b6e9e4f1b"
ACCOUNT_ID = "101-001-26404107-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [6]:
session = requests.Session()

In [8]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [17]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [25]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [26]:
response = session.get(url, params=None, data=None, headers=None)

In [27]:
response.status_code

200

In [29]:
data = response.json()

In [32]:
instruments_list = data['instruments']

In [33]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [34]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [36]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = {k: i[k] for k in key_i}

In [38]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [39]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [91]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()

    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']

        prices = ['mid', 'bid', 'ask']
        ohlc = ['o', 'h', 'l', 'c']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        
        final_data.append(new_dict)
    return pd.DataFrame.from_dict(final_data)

def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    if len(data) == 0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")

In [81]:
code, data = fetch_candles("EUR_USD", count=10, granularity="H4")
candles_df = get_candles_df(data)

In [92]:
create_data_file("EUR_USD", count=10, granularity="H4")

EUR_USD H4 9 candles, 2023-07-28 09:00:00+00:00 2023-07-31 17:00:00+00:00


In [93]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [94]:
instruments_dict.keys()

dict_keys(['EUR_ZAR', 'EUR_CHF', 'USD_PLN', 'GBP_AUD', 'EUR_AUD', 'EUR_SGD', 'NZD_CAD', 'TRY_JPY', 'CAD_HKD', 'USD_CHF', 'AUD_HKD', 'NZD_CHF', 'AUD_CHF', 'GBP_CHF', 'USD_THB', 'GBP_CAD', 'EUR_HKD', 'CHF_JPY', 'GBP_HKD', 'EUR_NZD', 'AUD_SGD', 'EUR_JPY', 'EUR_TRY', 'USD_JPY', 'SGD_JPY', 'GBP_ZAR', 'ZAR_JPY', 'AUD_JPY', 'USD_ZAR', 'CAD_JPY', 'NZD_HKD', 'USD_CZK', 'USD_DKK', 'USD_SEK', 'GBP_SGD', 'EUR_DKK', 'CHF_ZAR', 'CAD_CHF', 'GBP_USD', 'USD_MXN', 'USD_HUF', 'USD_HKD', 'EUR_USD', 'EUR_CAD', 'AUD_CAD', 'GBP_PLN', 'EUR_PLN', 'GBP_NZD', 'EUR_HUF', 'EUR_NOK', 'CHF_HKD', 'EUR_GBP', 'AUD_NZD', 'CAD_SGD', 'EUR_CZK', 'NZD_JPY', 'USD_TRY', 'GBP_JPY', 'SGD_CHF', 'USD_CNH', 'USD_NOK', 'NZD_SGD', 'NZD_USD', 'USD_CAD', 'AUD_USD', 'HKD_JPY', 'EUR_SEK', 'USD_SGD'])

In [95]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4001, granularity=g)

EUR_USD H1 4000 candles, 2022-12-08 07:00:00+00:00 2023-07-31 22:00:00+00:00
EUR_USD H4 4000 candles, 2021-01-07 06:00:00+00:00 2023-07-31 17:00:00+00:00
EUR_GBP H1 4000 candles, 2022-12-08 07:00:00+00:00 2023-07-31 22:00:00+00:00
EUR_GBP H4 4000 candles, 2021-01-07 06:00:00+00:00 2023-07-31 17:00:00+00:00
EUR_JPY H1 4000 candles, 2022-12-08 07:00:00+00:00 2023-07-31 22:00:00+00:00
EUR_JPY H4 4000 candles, 2021-01-06 22:00:00+00:00 2023-07-31 17:00:00+00:00
EUR_CHF H1 4000 candles, 2022-12-08 07:00:00+00:00 2023-07-31 22:00:00+00:00
EUR_CHF H4 4000 candles, 2021-01-07 06:00:00+00:00 2023-07-31 17:00:00+00:00
EUR_NZD H1 4000 candles, 2022-12-08 06:00:00+00:00 2023-07-31 22:00:00+00:00
EUR_NZD H4 4000 candles, 2021-01-07 06:00:00+00:00 2023-07-31 17:00:00+00:00
EUR_CAD H1 4000 candles, 2022-12-08 07:00:00+00:00 2023-07-31 22:00:00+00:00
EUR_CAD H4 4000 candles, 2021-01-07 06:00:00+00:00 2023-07-31 17:00:00+00:00
EUR_AUD H1 4000 candles, 2022-12-08 06:00:00+00:00 2023-07-31 22:00:00+00:00